# Interactive time-gating

## Preliminaries

### Imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import interactive_gating_with_unc_utils as utils

base = utils.BaseMethods()

### Compare different available datasets

In [ ]:
base.compare_different_datasets()

## Time Gating

### Load and Visualize Raw Data

In [ ]:
# data to be used for further processing
data_raw = f, s11_ri, s11_ri_cov = base.load_data("empirical_cov")

# get corresponding time
Nx = len(s11_ri) - 1
t_span = 1 / np.mean(np.diff(f))  # original f, not f_mod
t = np.linspace(0, t_span, num=Nx)


### Adjust Time Gating Process Settings

In [ ]:
# define window and gate
w, uw = base.window(size=len(f), kind="neutral")
gate = lambda t: base.gate(t, t_start=0.0, t_end=0.18, kind="kaiser", order=2.5*np.pi)

# store settings in dicts
data = {"f": f, "s_ri": s11_ri, "s_ri_cov": s11_ri_cov}
config = {
    "window": {"val": w, "cov": uw},
    "zeropad": {"pad_len": 2500, "Nx": Nx},
    "gate": {"gate_func": gate, "time": t},
    "renormalization": None,
}

### Perform the Gating Using Two Different Approaches

In [ ]:
s_gated_ri, s11_gated_ri_cov = base.perform_time_gating_method_1(data, config)
s_gated_mcconv_ri, s11_gated_mcconv_ri_cov = base.perform_time_gating_method_2(data, config)

## Visualiziations

### In the Frequency Domain (Raw, Method1, Method2, Renorm)

### In the Time Domain (Show Gate Position, Unc. in Time Domain)

### CHECK WHICH OTHER FIGURES ARE OF IMPORTANCE + WHAT INTERNAL INFORMATION NEEDS TO BE RETURNABLE

In [ ]:
# visualize raw input in the frequency domain

args_raw = {"l": "raw", "c": "tab:red"}
plotdata = [[data_raw, args_raw],]

base.mag_phase_plot(plotdata)

#### visualize

In [ ]:
# visualize time domain data
fig_in_td, ax_in_td = plt.subplots(nrows=2, figsize=(8, 6), tight_layout=True, sharex=True)

ax_in_td[0].get_shared_x_axes().join(ax_in_td[0], ax_in_td[1])
ax_in_td[0].plot(t_shifted, np.abs(S11_shifted), label="S11", color="tab:green")
ax_in_td[0].plot(t_shifted_mod, np.abs(S11_shifted_mod), label="S11 mod", color="tab:gray")

ax_in_td[1].semilogy(t_shifted, np.sqrt(np.diag(S11_cov_shifted)), label="S11 unc", color="tab:green")
ax_in_td[1].semilogy(t_shifted_mod, np.sqrt(np.diag(S11_cov_shifted_mod)), label="S11 mod unc", color="tab:gray")

ax_in_td[0].legend()
ax_in_td[0].set_title("Time Domain")
ax_in_td[0].set_ylabel("signal magnitude [-]")
ax_in_td[1].set_ylabel("signal unc [-]")
ax_in_td[1].set_xlabel("t [ns]")

In [ ]:
# visualize time domain data
fig_in_td_zoom, ax_in_td_zoom = plt.subplots(nrows=2, figsize=(8, 6), tight_layout=True, sharex=True)

ax_in_td_zoom[0].get_shared_x_axes().join(ax_in_td[0], ax_in_td[1])
#ax_in_td_zoom[0].plot(t_shifted, np.abs(S11_shifted), label="S11", color="tab:green")
ax_in_td_zoom[0].plot(t_shifted_mod, np.abs(S11_shifted_mod), label="S11 mod", color="tab:gray")

#ax_in_td_zoom[1].plot(t_shifted, np.sqrt(np.diag(S11_cov_shifted)), label="S11 unc", color="tab:green")
ax_in_td_zoom[1].plot(t_shifted_mod, np.sqrt(np.diag(S11_cov_shifted_mod)), label="S11 mod unc", color="tab:gray")

ax_in_td_zoom[0].set_xlim((-0.1,0.65))

ax_in_td_zoom[0].legend()
ax_in_td_zoom[0].set_title("Time Domain")
ax_in_td_zoom[0].set_ylabel("signal magnitude [-]")
ax_in_td_zoom[1].set_ylabel("signal unc [-]")
ax_in_td_zoom[1].set_xlabel("t [ns]")

In [ ]:
if False:
    array_export = np.c_[t_shifted_mod, S11_shifted_mod, np.sqrt(np.diag(S11_cov_shifted_mod))]
    df_export = pandas.DataFrame(array_export)
    df_export.columns = ["time", "signal", "signal_unc"]
    df_export.to_excel("export.xlsx")

In [ ]:
# visualize time domain data
fig_in_td_cov, ax_in_td_cov = plt.subplots(nrows=1, figsize=(6, 6), tight_layout=True)
maxi = np.max(np.abs(S11_cov))
cnorm = colors.SymLogNorm(vmin=-maxi, vmax=maxi, linthresh=1e-14)

img0 = ax_in_td_cov.imshow(S11_cov_shifted_mod,  cmap="PuOr", norm=cnorm)

fig_in_td_cov.colorbar(img0, ax=ax_in_td_cov)
ax_in_td_cov.set_title("Covariance of (modified) time signal")

## Define the Time-Gate

In [ ]:
gate_array_shifted_mod, gate_unc_array_shifted_mod = utils.gate(t_shifted_mod, t_start=0.0, t_end=0.18, kind="kaiser", order=2.5*np.pi)
gate_cov_array_shifted_mod = np.diag(np.square(gate_unc_array_shifted_mod))

# "unshift" for later calculations
gate_array_mod, gate_cov_array_mod = shift_uncertainty(gate_array_shifted_mod, gate_cov_array_shifted_mod, shift=-(Nx_mod-1)//2)

In [ ]:
fig_gate, ax_gate = plt.subplots(nrows=2, figsize=(8, 8), tight_layout=True, sharex=True)

# setup secondary y-axis
ax2_gate = [None, None]
ax2_gate[0] = ax_gate[0].twinx()
ax2_gate[1] = ax_gate[1].twinx()

# plot signal and gate
ax_gate[0].plot(t_shifted_mod, np.abs(S11_shifted_mod), label="S11 mod", color="tab:gray")
ax2_gate[0].plot(t_shifted_mod, gate_array_shifted_mod, label="gate", color="r")

# plot uncertainty of signal and gate
ax_gate[1].plot(t_shifted_mod, np.sqrt(np.diag(S11_cov_shifted_mod)), label="S11 mod unc", color="tab:gray")
ax2_gate[1].plot(t_shifted_mod, gate_unc_array_shifted_mod, label="gate unc", color="r")

# decorate
ax_gate[0].set_title("Time Domain (zoomed)")
ax_gate[0].legend(loc="upper right")
ax2_gate[0].legend(loc="upper center")
ax_gate[0].set_xlim((-0.1,1.5))
ax2_gate[1].set_ylim(ax_gate[1].get_ylim())

ax_gate[0].set_ylabel("signal magnitude [-]")
ax2_gate[0].set_ylabel("gate [-]")
ax_gate[1].set_ylabel("signal unc [-]")
ax2_gate[1].set_ylabel("gate unc [-]")
ax_gate[1].set_xlabel("t [ns]")

ax_gate[0].yaxis.get_label().set_color("tab:gray")
ax2_gate[0].yaxis.get_label().set_color("r")
ax_gate[1].yaxis.get_label().set_color("tab:gray")
ax2_gate[1].yaxis.get_label().set_color("r")

## Apply the Time-Gate to the modified Signal

### Method 1 (multiplication and then DFT)
Applies the gate already in the time-domain and converts the result back to frequency domain. 

In [ ]:
# main calls
S11_gated_mod = S11_mod * gate_array_mod
S11_gated_cov_mod = np.diag(gate_array_mod) @ S11_cov_mod @ np.diag(gate_array_mod).T + np.diag(S11_mod) @ gate_cov_array_mod @ np.diag(S11_mod).T
s11_gated_ri_mod, s11_gated_ri_cov_mod = GUM_DFT(S11_gated_mod, S11_gated_cov_mod)

In [ ]:
# undo zero-padding
s11_gated_ri = trimOrPad(s11_gated_ri_mod, length=len(f), real_imag_type=True) * Nx / Nx_mod
s11_gated_ri_cov = trimOrPad(s11_gated_ri_cov_mod, length=len(f), real_imag_type=True) * Nx / Nx_mod

# undo windowing
s11_gated_ri, s11_gated_ri_cov = utils.apply_window(s11_gated_ri, 1.0 / window, s11_gated_ri_cov, None)

#### Visualize Time Gated s11-Parameter

In [ ]:
# visualize result amp/phase
NN=len(s11_gated_ri)//2

# convert back to amplitude/phase representation
s11_gated_A, s11_gated_P, s11_gated_UAP = DFT2AmpPhase(s11_gated_ri, s11_gated_ri_cov)
s11_gated_UA = np.sqrt(np.diag(s11_gated_UAP)[:NN])
s11_gated_UP = np.sqrt(np.diag(s11_gated_UAP)[NN:])

fig_in, ax_in = plt.subplots(nrows=4, figsize=(8, 8), tight_layout=True)
ax_in[0].plot(f, s11_gated_A, label="s11 gated", color="tab:blue")
ax_in[1].plot(f, s11_gated_UA, label="s11 gated", color="tab:blue")
ax_in[2].plot(f, np.rad2deg(s11_gated_P), label="s11 gated", color="tab:blue")
ax_in[3].plot(f, np.rad2deg(s11_gated_UP), label="s11 gated", color="tab:blue")

ax_in[0].legend()
ax_in[0].grid(which="both", axis="both")
ax_in[0].set_title("Frequency Domain")
ax_in[0].set_ylabel("magnitude [-]")
ax_in[1].set_ylabel("magnitude unc [-]")
ax_in[2].set_ylabel("phase [°]")
ax_in[3].set_ylabel("phase unc [°]")
ax_in[3].set_xlabel("f [GHz]")


### Method 2 (DFT and then complex convolution)

Transforms gate into frequency domain and applies gate to the original (frequency domain) signal by using convolution operation. Output should match first method up to numerical precision. (Done with uncertainty evaluation by Monte Carlo, analytical uncertainty evaluation out of scope for now.)

In [ ]:
# main calls
#gate_spectrum = np.fft.rfft(gate_array)
#s11_gated_conv = complex_convolution_of_two_half_spectra(ri2c(s11_ri), gate_spectrum)

# Monte Carlo of this main call

# draw gate and signal
def draw_samples(size, x1, x1_cov, x2, x2_cov):
    SAMPLES_X1 = np.random.multivariate_normal(x1, x1_cov, size)
    SAMPLES_X2 = np.random.multivariate_normal(x2, x2_cov, size)
    return (SAMPLES_X1, SAMPLES_X2)

# evaluate
n_runs = 100
results = []
for s11_ri_mc, gate_array_mc in zip(*draw_samples(size=n_runs, x1=s11_ri_mod, x1_cov=s11_ri_cov_mod, x2=gate_array_mod, x2_cov=gate_cov_array_mod)):

    # explicitly zero pad
    s11_ri_mc = trimOrPad(s11_ri_mc, length=len(f) + pad_len//2, real_imag_type=True) * Nx_mod / Nx

    # main call
    gate_spectrum_tmp = np.fft.rfft(gate_array_mc)
    s11_gated_conv_tmp = complex_convolution_of_two_half_spectra(ri2c(s11_ri_mc), gate_spectrum_tmp)

    # undo zero-padding
    s11_gated_conv_tmp = trimOrPad(s11_gated_conv_tmp, length=len(f), real_imag_type=False) * Nx / Nx_mod

    # undo windowing
    s11_gated_conv_tmp = s11_gated_conv_tmp / window
    
    # save result
    results.append(c2ri(s11_gated_conv_tmp))

# extract mean and covariance
s11_gated_mcconv_ri = np.mean(results, axis=0)
s11_gated_mcconv_ri_cov = np.cov(results, rowvar=False)
s11_gated_mcconv = ri2c(s11_gated_mcconv_ri)

#### Visualize Time Gated s11-Parameter

In [ ]:
# visualize result amp/phase
NN=len(s11_gated_mcconv_ri)//2

# convert back to amplitude/phase representation
s11_gated_mcconv_A, s11_gated_mcconv_P, s11_gated_mcconv_UAP = DFT2AmpPhase(s11_gated_mcconv_ri, s11_gated_mcconv_ri_cov)
s11_gated_mcconv_UA = np.sqrt(np.diag(s11_gated_mcconv_UAP)[:NN])
s11_gated_mcconv_UP = np.sqrt(np.diag(s11_gated_mcconv_UAP)[NN:])

fig_in, ax_in = plt.subplots(nrows=4, figsize=(8, 8), tight_layout=True)
ax_in[0].plot(f, s11_gated_mcconv_A, label="s11 gated", color="tab:orange")
ax_in[1].plot(f, s11_gated_mcconv_UA, label="s11 gated", color="tab:orange")
ax_in[2].plot(f, np.rad2deg(s11_gated_mcconv_P), label="s11 gated", color="tab:orange")
ax_in[3].plot(f, np.rad2deg(s11_gated_mcconv_UP), label="s11 gated", color="tab:orange")

ax_in[0].legend()
ax_in[0].grid(which="both", axis="both")
ax_in[0].set_title("Frequency Domain")
ax_in[0].set_ylabel("magnitude [-]")
ax_in[1].set_ylabel("magnitude unc [-]")
ax_in[2].set_ylabel("phase [°]")
ax_in[3].set_ylabel("phase unc [°]")
ax_in[3].set_xlabel("f [GHz]")

## Comparison

### Original and Time-Gated Spectra of both methods

In [ ]:
fig_in, ax_in = plt.subplots(nrows=4, figsize=(8, 10), tight_layout=True)

ax_in[0].plot(f, s11_mag, label="s11 orig", color="tab:gray")
ax_in[0].plot(f, s11_gated_A, label="s11 gated", color="tab:blue", linewidth=5)
ax_in[0].plot(f, np.abs(s11_gated_mcconv), label="s11 gated (MC conv)", color="tab:orange")

ax_in[1].plot(f, s11_mag_unc, label="s11 orig unc", color="tab:gray")
ax_in[1].plot(f, s11_gated_UA, label="s11 gated", color="tab:blue", linewidth=5)
ax_in[1].plot(f, s11_gated_mcconv_UA, label="s11 gated (MC conv)", color="tab:orange")

ax_in[2].plot(f, np.rad2deg(s11_phase), label="s11 orig", color="tab:gray")
ax_in[2].plot(f, np.rad2deg(s11_gated_P), label="s11 gated", color="tab:blue", linewidth=5)
ax_in[2].plot(f, np.rad2deg(np.angle(s11_gated_mcconv)), label="s11 gated (MC conv)", color="tab:orange")

ax_in[3].plot(f, np.rad2deg(s11_phase_unc), label="s11 orig unc", color="tab:gray")
ax_in[3].plot(f, np.rad2deg(s11_gated_UP), label="s11 gated", color="tab:blue", linewidth=5)
ax_in[3].plot(f, np.rad2deg(s11_gated_mcconv_UP), label="s11 gated (MC conv)", color="tab:orange")


ax_in[0].legend()
ax_in[1].legend()
ax_in[2].legend()
ax_in[3].legend()
ax_in[0].set_title("Frequency Domain")
ax_in[0].set_ylabel("magnitude [-]")
ax_in[1].set_ylabel("magnitude unc [-]")
ax_in[2].set_ylabel("phase [°]")
ax_in[3].set_ylabel("phase unc [°]")
ax_in[3].set_xlabel("f [GHz]")
ax_in[1].set_yscale("log")
ax_in[3].set_yscale("log")
ax_in[3].set_ylim(1e-3, 1.5e1)

### Covariance Matrices

Covariances in Re/Im Representation of:

- input data
- output of method 1 (analytical uncertainty evaluation)
- output of method 2 (Monte Carlo uncertainty evaluation)

In [ ]:
# utility function for annotating plots
def annotate_real_imag_plot(ax, annotate_x=True, annotate_y=True):
    # define new tick positions
    labels_re = [0, 10, 20, 30]  # GHz
    labels_re = [0, 8, 16, 24]  # GHz
    labels_im = labels_re
    labels = labels_re + labels_im

    # define new labels for these positions
    ticks_re = [np.flatnonzero(f == l).item() for l in labels_re]
    ticks_im = [f.size + 1 + k for k in ticks_re]
    ticks = ticks_re + ticks_im

    # define colors for the labels to distinguish real and imag parts
    colors_re = ["k"] * len(ticks_re)
    colors_im = ["r"] * len(ticks_im)
    tick_colors = colors_re + colors_im

    if annotate_x:
        # xticks (label, position, color)
        ax.set_xticks(ticks=ticks, labels=labels)
        for ticklabel, c in zip(ax.get_xticklabels(), tick_colors):
            ticklabel.set_color(c)

        # axis label
        ax.set_xlabel("frequency [GHz]")

        # nice brace real part
        ax.annotate(
            "real",
            xy=(0.25, -0.01),
            xytext=(0.25, -0.10),
            fontsize=14,
            ha="center",
            va="bottom",
            xycoords="axes fraction",
            color="black",
            #arrowprops=dict(arrowstyle="-[, widthB=6.0, lengthB=.5"),
        )

        # nice brace imag part
        ax.annotate(
            "imag",
            xy=(0.75, -0.01),
            xytext=(0.75, -0.10),
            fontsize=14,
            ha="center",
            va="bottom",
            xycoords="axes fraction",
            color="red",
            #arrowprops=dict(arrowstyle="-[, widthB=6.0, lengthB=.5"),
        )

    if annotate_y:
        # yticks (label, position, color)
        ax.set_yticks(ticks=ticks, labels=labels)
        for ticklabel, c in zip(ax.get_yticklabels(), tick_colors):
            ticklabel.set_color(c)

        # axis label
        ax.set_ylabel("frequency [GHz]")

        # nice brace real part
        ax.annotate(
            "real",
            xy=(-0.01, 0.75),
            xytext=(-0.10, 0.75),
            fontsize=14,
            ha="left",
            va="center",
            xycoords="axes fraction",
            rotation=90,
            color="black",
            #arrowprops=dict(arrowstyle="-[, widthB=6.0, lengthB=.5"),
        )

        # nice brace imag part
        ax.annotate(
            "imag",
            xy=(-0.01, 0.25),
            xytext=(-0.10, 0.25),
            fontsize=14,
            ha="left",
            va="center",
            xycoords="axes fraction",
            rotation=90,
            color="red",
            #arrowprops=dict(arrowstyle="-[, widthB=6.0, lengthB=.5"),
        )

    return ax


In [ ]:
fig_cov, ax_cov = plt.subplots(nrows=3, figsize=(8, 25), tight_layout=True)
mini = max(1e-11, min(np.abs(s11_ri_cov).min(), np.abs(s11_gated_ri_cov).min()))
maxi = min(np.abs(s11_ri_cov).max(), np.abs(s11_gated_ri_cov).max())

cnorm = colors.SymLogNorm(vmin=-maxi, vmax=maxi, linthresh=mini)
img1 = ax_cov[0].imshow(s11_ri_cov, cmap="PuOr", norm=cnorm)
img2 = ax_cov[1].imshow(s11_gated_ri_cov, cmap="PuOr", norm=cnorm)
img3 = ax_cov[2].imshow(s11_gated_mcconv_ri_cov, cmap="PuOr", norm=cnorm)
fig_cov.colorbar(img1, ax=ax_cov[0])
fig_cov.colorbar(img2, ax=ax_cov[1])
fig_cov.colorbar(img3, ax=ax_cov[2])

ax_cov[0].set_title("Covariance of s11_ri")
ax_cov[0] = annotate_real_imag_plot(ax_cov[0])

ax_cov[1].set_title("Covariance of s11_gated_ri")
ax_cov[1] = annotate_real_imag_plot(ax_cov[1])

ax_cov[2].set_title("Covariance of s11_gated_mcconv_ri_cov")
ax_cov[2] = annotate_real_imag_plot(ax_cov[2])

#### Compare Gated Spectra and Covariance Matrices of analytical and Monte Carlo approach

In [ ]:
# visual comparison of the two methods:
fig_comp, ax_comp = plt.subplots(nrows=3, figsize=(8, 20), tight_layout=True)

# mean signed difference of values
ax_comp[0].plot(s11_gated_ri - s11_gated_mcconv_ri)
ax_comp[0].set_title("Mean Signed Difference of Gated Spectra")
ax_comp[0] = annotate_real_imag_plot(ax_comp[0], annotate_y=False)

# mean signed difference of covariance matrices
img4 = ax_comp[1].imshow(s11_gated_ri_cov - s11_gated_mcconv_ri_cov, cmap="PuOr", norm=cnorm)
fig_comp.colorbar(img4, ax=ax_comp[1])

ax_comp[1].set_title("Signed Difference of both Covariance Matrices")
ax_comp[1] = annotate_real_imag_plot(ax_comp[1])

# Kullback-Leibler divergence of covariance matrices
kl_div = special.kl_div(s11_gated_ri_cov, s11_gated_mcconv_ri_cov)
cnorm_kl = colors.LogNorm(vmin=1e-12, vmax=1e-8, clip=True)

img5 = ax_comp[2].imshow(kl_div, cmap="binary", norm=cnorm_kl)
fig_comp.colorbar(img5, ax=ax_comp[2])


ax_comp[2].set_title("Kullback-Leibler divergence of both Covariance Matrices")
ax_comp[2] = annotate_real_imag_plot(ax_comp[2])